### Data Cleaning

In [246]:
##import das bibliotecas e adequando colunas, linhas e formato de números

from google.cloud import bigquery
from dotenv import load_dotenv
load_dotenv()

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import re

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Temp\desafiolh-445818-3cb0f62cb9ef.json"


# Configurar Pandas para exibir todas as colunas e todas as linhas completas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)


pd.options.display.float_format = '{:.2f}'.format


In [247]:
# Configurar o cliente do BigQuery
client = bigquery.Client()

# Nome do dataset e tabela
dataset_id = 'raw_data'




In [248]:
# Listar tabelas no dataset
tables = client.list_tables('raw_data')
print("Tabelas disponíveis:")
for table in tables:
    print(table.table_id)




Tabelas disponíveis:


humanresources_employee
person_address
person_businessentity
person_person
production_location
production_product
production_productcategory
production_productinventory
production_productsubcategory
sales_customer
sales_salesorderdetail
sales_salesorderheader
sales_salesterritory
sales_store


In [249]:
query = f"SELECT * FROM `raw_data.humanresources_employee`"
data = client.query(query).result().to_dataframe()

# Expandir a coluna JSON
raw_data = pd.json_normalize(data['data'])

# Exibir os dados expandidos
print(raw_data.sample(n=50))


c:\Users\wrpen\OneDrive\Desktop\df_lh\venv\lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
c:\Users\wrpen\OneDrive\Desktop\df_lh\venv\lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:208: UserWarning: Unable to determine Arrow type for field 'data'.
  warnings.warn(


       birthdate  businessentityid  currentflag gender    hiredate                           jobtitle                       loginid maritalstatus                modifieddate nationalidnumber organizationnode                               rowguid  salariedflag  sickleavehours  vacationhours
817   1984-08-01               238         True      M  2009-01-01                          Recruiter        adventure-works\vidur0             S         2014-06-30T00:00:00        264306399          /4/1/3/  47e76b59-b137-4572-ad55-6b3e68c13bd3         False              45             50
1399  1976-02-11                67         True      M  2009-03-05       Production Technician - WC60          adventure-works\jay0             S         2014-06-30T00:00:00         63179277        /3/1/5/5/  18187809-5294-474f-85a1-d5be6bd518d6         False              36             32
1679  1984-10-11               249         True      F  2008-12-25                    Finance Manager        adventure-works\we

### Verificando valores nulos/em branco nos dados

In [250]:
#valores nulos

raw_data.isnull().sum()

birthdate           0
businessentityid    0
currentflag         0
gender              0
hiredate            0
jobtitle            0
loginid             0
maritalstatus       0
modifieddate        0
nationalidnumber    0
organizationnode    0
rowguid             0
salariedflag        0
sickleavehours      0
vacationhours       0
dtype: int64

In [251]:
#total valores unicos de cada variável

valores_unicos = []

for i in raw_data.columns[0:15].tolist():
    print(i, ':', len(raw_data[i].astype(str).value_counts()))
    valores_unicos.append(len(raw_data[i].astype(str).value_counts()))

birthdate : 275
businessentityid : 290
currentflag : 1
gender : 2
hiredate : 164
jobtitle : 67
loginid : 290
maritalstatus : 2
modifieddate : 2
nationalidnumber : 290
organizationnode : 290
rowguid : 290
salariedflag : 2
sickleavehours : 51
vacationhours : 100


In [252]:
# Identificar duplicatas com base em 'businessentityid'
duplicatas = raw_data[raw_data.duplicated(subset=['businessentityid'], keep=False)]

# Verificar se existem duplicatas
if not duplicatas.empty:
    # Ordenar duplicatas por 'businessentityid' e 'modifieddate'
    duplicatas_ordenadas = duplicatas.sort_values(by=['businessentityid', 'modifieddate'])
    
    # Exibir duplicatas ordenadas
    print("Duplicatas ordenadas:")
    print(duplicatas_ordenadas)
else:
    print("Não foram encontradas duplicatas.")


Duplicatas ordenadas:
       birthdate  businessentityid  currentflag gender    hiredate                                  jobtitle                       loginid maritalstatus                modifieddate nationalidnumber organizationnode                               rowguid  salariedflag  sickleavehours  vacationhours
0     1969-01-29                 1         True      M  2009-01-14                   Chief Executive Officer          adventure-works\ken0             S         2014-06-30T00:00:00        295847284                /  f01251e5-96a3-448d-981e-0f99d789110d          True              69             99
290   1969-01-29                 1         True      M  2009-01-14                   Chief Executive Officer          adventure-works\ken0             S         2014-06-30T00:00:00        295847284                /  f01251e5-96a3-448d-981e-0f99d789110d          True              69             99
580   1969-01-29                 1         True      M  2009-01-14                  

In [253]:
print(duplicatas_ordenadas.drop_duplicates())


      birthdate  businessentityid  currentflag gender    hiredate                                  jobtitle                       loginid maritalstatus                modifieddate nationalidnumber organizationnode                               rowguid  salariedflag  sickleavehours  vacationhours
0    1969-01-29                 1         True      M  2009-01-14                   Chief Executive Officer          adventure-works\ken0             S         2014-06-30T00:00:00        295847284                /  f01251e5-96a3-448d-981e-0f99d789110d          True              69             99
1    1971-08-01                 2         True      F  2008-01-31             Vice President of Engineering        adventure-works\terri0             S         2014-06-30T00:00:00        245797967              /1/  45e8f437-670d-4409-93cb-f9424a40d6ee          True              20              1
3    1974-11-12                 3         True      M  2007-11-11                       Engineering Manager  

In [254]:
duplicados_businessentityid = raw_data[raw_data.duplicated(subset=['businessentityid'], keep=False)]

# Ordenar por 'businessentityid' para facilitar a análise
duplicados_ordenados = duplicados_businessentityid.sort_values(by=['businessentityid'])

# Exibir todas as linhas duplicadas
print(duplicados_ordenados)


       birthdate  businessentityid  currentflag gender    hiredate                                  jobtitle                       loginid maritalstatus                modifieddate nationalidnumber organizationnode                               rowguid  salariedflag  sickleavehours  vacationhours
0     1969-01-29                 1         True      M  2009-01-14                   Chief Executive Officer          adventure-works\ken0             S         2014-06-30T00:00:00        295847284                /  f01251e5-96a3-448d-981e-0f99d789110d          True              69             99
1308  1969-01-29                 1         True      M  2009-01-14                   Chief Executive Officer          adventure-works\ken0             S         2014-06-30T00:00:00        295847284                /  f01251e5-96a3-448d-981e-0f99d789110d          True              69             99
1310  1969-01-29                 1         True      M  2009-01-14                   Chief Executive Offic

In [255]:
pd.set_option('display.max_rows', None)
contagem = raw_data['businessentityid'].value_counts()

# Filtrar apenas os IDs que aparecem mais de uma vez
repetidos = contagem[contagem > 1]

# Exibir repetidos novamente
print(repetidos)

businessentityid
290    7
1      7
212    7
216    7
215    7
220    7
222    7
225    7
227    7
228    7
197    7
194    7
195    7
193    7
204    7
198    7
218    7
209    7
189    7
187    7
183    7
185    7
186    7
179    7
191    7
190    7
152    7
161    7
163    7
160    7
165    7
173    7
164    7
176    7
80     7
79     7
91     7
83     7
82     7
88     7
89     7
90     7
61     7
71     7
65     7
68     7
67     7
70     7
63     7
78     7
59     7
55     7
57     7
54     7
77     7
60     7
66     7
76     7
40     7
46     7
52     7
49     7
51     7
50     7
47     7
53     7
150    7
147    7
144    7
149    7
155    7
142    7
148    7
158    7
140    7
135    7
136    7
137    7
126    7
141    7
143    7
154    7
102    7
103    7
109    7
111    7
123    7
118    7
138    7
134    7
92     7
85     7
86     7
106    7
104    7
105    7
108    7
101    7
196    7
199    7
206    7
208    7
207    7
205    7
217    7
211    7
181    7
180    7
177    7
20

In [256]:
#copia da humanresources_employee
raw_data_bkp = raw_data.copy()

# Ordenar o DataFrame por 'businessentityid' e 'modifieddate'
raw_data = raw_data.sort_values(by=['businessentityid', 'modifieddate'])

# Remover duplicatas mantendo a última ocorrência com base em 'modifieddate'
raw_data = raw_data.drop_duplicates(subset=['businessentityid'], keep='last')

print(f"Linhas após remover duplicatas (baseando-se na última 'modifieddate'): {len(raw_data)}")


Linhas após remover duplicatas (baseando-se na última 'modifieddate'): 290


In [257]:
# Verificar informações do DataFrame
print(raw_data.info())


<class 'pandas.core.frame.DataFrame'>
Index: 290 entries, 1886 to 2022
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   birthdate         290 non-null    object
 1   businessentityid  290 non-null    int64 
 2   currentflag       290 non-null    bool  
 3   gender            290 non-null    object
 4   hiredate          290 non-null    object
 5   jobtitle          290 non-null    object
 6   loginid           290 non-null    object
 7   maritalstatus     290 non-null    object
 8   modifieddate      290 non-null    object
 9   nationalidnumber  290 non-null    object
 10  organizationnode  290 non-null    object
 11  rowguid           290 non-null    object
 12  salariedflag      290 non-null    bool  
 13  sickleavehours    290 non-null    int64 
 14  vacationhours     290 non-null    int64 
dtypes: bool(2), int64(3), object(10)
memory usage: 32.3+ KB
None


In [258]:
# Identificar colunas com datas
date_columns = ['birthdate', 'hiredate', 'modifieddate']

# Converter as colunas para datetime se ainda não estiverem
for col in date_columns:
    raw_data[col] = pd.to_datetime(raw_data[col], errors='coerce')

# Criar uma cópia do DataFrame para exibição formatada
formatted_data = raw_data.copy()

# Formatar todas as colunas de datas para exibição
for col in date_columns:
    formatted_data[col] = raw_data[col].dt.strftime('%Y-%m-%d %H:%M:%S')

# Exibir o DataFrame formatado
print(formatted_data.head())

# Verificar os tipos originais permanecem datetime64[ns]
print("\nTipos originais das colunas no DataFrame principal:")
print(raw_data[date_columns].dtypes)


                birthdate  businessentityid  currentflag gender             hiredate                       jobtitle                   loginid maritalstatus         modifieddate nationalidnumber organizationnode                               rowguid  salariedflag  sickleavehours  vacationhours
1886  1969-01-29 00:00:00                 1         True      M  2009-01-14 00:00:00        Chief Executive Officer      adventure-works\ken0             S  2014-06-30 00:00:00        295847284                /  f01251e5-96a3-448d-981e-0f99d789110d          True              69             99
1887  1971-08-01 00:00:00                 2         True      F  2008-01-31 00:00:00  Vice President of Engineering    adventure-works\terri0             S  2014-06-30 00:00:00        245797967              /1/  45e8f437-670d-4409-93cb-f9424a40d6ee          True              20              1
1742  1974-11-12 00:00:00                 3         True      M  2007-11-11 00:00:00            Engineering Manager  a

In [259]:
# Padronizar textos em jobtitle e gender
raw_data['jobtitle'] = raw_data['jobtitle'].str.strip().str.title()
raw_data['gender'] = raw_data['gender'].str.strip().str.upper()

# Verificar valores únicos para garantir a padronização
print("Valores únicos em 'jobtitle':", raw_data['jobtitle'].unique())
print("Valores únicos em 'gender':", raw_data['gender'].unique())


Valores únicos em 'jobtitle': ['Chief Executive Officer' 'Vice President Of Engineering'
 'Engineering Manager' 'Senior Tool Designer' 'Design Engineer'
 'Research And Development Manager' 'Research And Development Engineer'
 'Tool Designer' 'Senior Design Engineer' 'Marketing Manager'
 'Marketing Assistant' 'Marketing Specialist'
 'Vice President Of Production' 'Production Control Manager'
 'Production Supervisor - Wc60' 'Production Technician - Wc60'
 'Production Supervisor - Wc10' 'Production Technician - Wc10'
 'Production Supervisor - Wc50' 'Production Technician - Wc50'
 'Production Supervisor - Wc30' 'Production Technician - Wc30'
 'Production Supervisor - Wc40' 'Production Technician - Wc40'
 'Shipping And Receiving Supervisor' 'Stocker'
 'Shipping And Receiving Clerk' 'Production Supervisor - Wc20'
 'Production Technician - Wc20' 'Production Supervisor - Wc45'
 'Production Technician - Wc45' 'Quality Assurance Manager'
 'Quality Assurance Supervisor' 'Quality Assurance Technic

In [260]:
# Identificar colunas numéricas para análise 
numeric_columns = ['sickleavehours', 'vacationhours']

# Exibir estatísticas descritivas
print(raw_data[numeric_columns].describe())

# Calcular limites para outliers (IQR - Intervalo Interquartil)
for col in numeric_columns:
    q1 = raw_data[col].quantile(0.25)
    q3 = raw_data[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Exibir os limites
    print(f"\nColuna: {col}")
    print(f"Limite inferior: {lower_bound}, Limite superior: {upper_bound}")
    
    # Filtrar outliers
    outliers = raw_data[(raw_data[col] < lower_bound) | (raw_data[col] > upper_bound)]
    print(f"Outliers detectados ({len(outliers)}):")
    print(outliers[[col]])


       sickleavehours  vacationhours
count          290.00         290.00
mean            45.31          50.61
std             14.54          28.79
min             20.00           0.00
25%             33.00          26.25
50%             46.00          51.00
75%             58.00          75.00
max             80.00          99.00

Coluna: sickleavehours
Limite inferior: -4.5, Limite superior: 95.5
Outliers detectados (0):
Empty DataFrame
Columns: [sickleavehours]
Index: []

Coluna: vacationhours
Limite inferior: -46.875, Limite superior: 148.125
Outliers detectados (0):
Empty DataFrame
Columns: [vacationhours]
Index: []


In [261]:

# Criar a coluna 'hire_year' com base em 'hiredate'
raw_data['hire_year'] = raw_data['hiredate'].dt.year


#verificando outros dados para detectar outliers
anos_contratacao = [int(ano) for ano in raw_data['hire_year'].unique()]
anos_contratacao.sort()
print(anos_contratacao)

print("Valores únicos em 'gender':", raw_data['gender'].unique())
print("Valores únicos em 'salariedflag':", raw_data['salariedflag'].unique())


[2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013]
Valores únicos em 'gender': ['M' 'F']
Valores únicos em 'salariedflag': [ True False]


In [262]:
# Verificar valores ausentes
missing = raw_data.isnull().sum()
print("Valores ausentes por coluna:")
print(missing)

# Tratar colunas críticas
if missing['hiredate'] > 0:
    print("Tratar valores ausentes em 'hiredate' (decisão: remover ou imputar)")

if missing['jobtitle'] > 0:
    print("Tratar valores ausentes em 'jobtitle' (decisão: remover ou imputar)")

if missing['businessentityid'] > 0:
    print("Erro crítico: 'businessentityid' não pode ter valores ausentes!")


Valores ausentes por coluna:
birthdate           0
businessentityid    0
currentflag         0
gender              0
hiredate            0
jobtitle            0
loginid             0
maritalstatus       0
modifieddate        3
nationalidnumber    0
organizationnode    0
rowguid             0
salariedflag        0
sickleavehours      0
vacationhours       0
hire_year           0
dtype: int64


In [263]:
# Verificar linhas com 'modifieddate' ausente
missing_modifieddate = raw_data[raw_data['modifieddate'].isnull()]
print("Linhas com 'modifieddate' ausente:")
print(missing_modifieddate)


Linhas com 'modifieddate' ausente:
      birthdate  businessentityid  currentflag gender   hiredate                    jobtitle                  loginid maritalstatus modifieddate nationalidnumber organizationnode                               rowguid  salariedflag  sickleavehours  vacationhours  hire_year
1747 1982-11-03                26         True      M 2008-12-01  Production Control Manager   adventure-works\peter0             M          NaT        277173473            /3/1/  69d5d162-e817-45e7-9dec-5d9b8310e7b1          True              41             43       2008
1981 1977-10-26               211         True      M 2009-02-28   Quality Assurance Manager   adventure-works\hazem0             S          NaT        398223854            /3/2/  05c84608-f445-4f9d-bb5c-0828c309c29d          True              60             80       2009
2002 1968-09-17               222         True      M 2008-12-12            Master Scheduler  adventure-works\ascott0             S          NaT  

In [264]:
# Preencher 'modifieddate' ausente ou igual a 'hiredate', pois pode ser a ultima data de modificação no sistema.
raw_data.loc[raw_data['modifieddate'].isnull() | (raw_data['modifieddate'] == pd.Timestamp('1900-01-01')), 'modifieddate'] = raw_data['hiredate']

# Exibir as linhas ajustadas
print("Linhas onde 'modifieddate' foi ajustado para 'hiredate':")
print(raw_data.loc[raw_data['modifieddate'] == raw_data['hiredate']])


Linhas onde 'modifieddate' foi ajustado para 'hiredate':
      birthdate  businessentityid  currentflag gender   hiredate                    jobtitle                  loginid maritalstatus modifieddate nationalidnumber organizationnode                               rowguid  salariedflag  sickleavehours  vacationhours  hire_year
1747 1982-11-03                26         True      M 2008-12-01  Production Control Manager   adventure-works\peter0             M   2008-12-01        277173473            /3/1/  69d5d162-e817-45e7-9dec-5d9b8310e7b1          True              41             43       2008
1981 1977-10-26               211         True      M 2009-02-28   Quality Assurance Manager   adventure-works\hazem0             S   2009-02-28        398223854            /3/2/  05c84608-f445-4f9d-bb5c-0828c309c29d          True              60             80       2009
2002 1968-09-17               222         True      M 2008-12-12            Master Scheduler  adventure-works\ascott0       

In [265]:
# Verificar unicidade de 'businessentityid'
is_unique = raw_data['businessentityid'].is_unique
print(f"'businessentityid' é único? {is_unique}")


'businessentityid' é único? True


In [266]:
# Definir regex para validar números (exemplo: apenas dígitos, 9 caracteres)
regex = r'^\d{9}$'

# Verificar valores inválidos
invalid_nationalid = raw_data[~raw_data['nationalidnumber'].astype(str).str.match(regex)]
print(f"Valores inválidos em 'nationalidnumber':\n{invalid_nationalid['nationalidnumber']}")


Valores inválidos em 'nationalidnumber':
1897    42487730
1749    56920285
1894    24756624
1890    52541318
1895    95958330
1755    72636981
1748    14417807
1771     6298838
1901    66073987
1926    33237992
1769     9659517
1960    10708100
1902    92096924
1803     8066363
1929    63179277
1800    36151748
1951    58791499
1918     1662732
1777     7201901
1962    90888098
1948    82638150
1768    54759846
1811     1300049
1764    45615666
1946    63761469
1813    25011600
1912    56772045
1773    97728960
2003    65848458
1979    60114406
1985    87268837
1836    19312190
1997    20244403
1822    28414965
1819       30845
2020    60517918
1877    20269531
2010    58317344
2008    61161660
1867    90836195
Name: nationalidnumber, dtype: object


In [267]:
regex_email = r'^[\w\.-]+@[\w\.-]+\.\w+$'
invalid_loginid = raw_data[~raw_data['loginid'].str.match(regex_email)]
print(f"Valores inválidos em 'loginid':\n{invalid_loginid['loginid']}")


Valores inválidos em 'loginid':
1886            adventure-works\ken0
1887          adventure-works\terri0
1742        adventure-works\roberto0
1888            adventure-works\rob0
1741           adventure-works\gail0
1740         adventure-works\jossef0
1743          adventure-works\dylan0
1889          adventure-works\diane1
1752           adventure-works\gigi0
1754        adventure-works\michael6
1896         adventure-works\ovidiu0
1756        adventure-works\thierry0
1750         adventure-works\janice0
1897        adventure-works\michael8
1749         adventure-works\sharon0
1894          adventure-works\david0
1898          adventure-works\kevin0
1899           adventure-works\john5
1890           adventure-works\mary2
1746         adventure-works\wanida0
1751          adventure-works\terry0
1895         adventure-works\sariya0
1744           adventure-works\mary0
1755           adventure-works\jill0
1893          adventure-works\james1
1747          adventure-works\peter0
1892  

In [268]:
is_unique = raw_data['nationalidnumber'].is_unique
print(f"'nationalidnumber' é único? {is_unique}")


'nationalidnumber' é único? True


In [269]:
# Criar um backup do DataFrame tratado
raw_data_bkp_v2 = raw_data.copy()

# Verificar o tamanho do backup e as primeiras linhas
print(f"Backup criado com {len(raw_data_bkp_v2)} linhas.")
print(raw_data_bkp_v2.head())


Backup criado com 290 linhas.
      birthdate  businessentityid  currentflag gender   hiredate                       jobtitle                   loginid maritalstatus modifieddate nationalidnumber organizationnode                               rowguid  salariedflag  sickleavehours  vacationhours  hire_year
1886 1969-01-29                 1         True      M 2009-01-14        Chief Executive Officer      adventure-works\ken0             S   2014-06-30        295847284                /  f01251e5-96a3-448d-981e-0f99d789110d          True              69             99       2009
1887 1971-08-01                 2         True      F 2008-01-31  Vice President Of Engineering    adventure-works\terri0             S   2014-06-30        245797967              /1/  45e8f437-670d-4409-93cb-f9424a40d6ee          True              20              1       2008
1742 1974-11-12                 3         True      M 2007-11-11            Engineering Manager  adventure-works\roberto0             M   2

In [270]:
# Verificar e documentar colunas existentes
print("Colunas mantidas no dataset:", raw_data.columns.tolist())


Colunas mantidas no dataset: ['birthdate', 'businessentityid', 'currentflag', 'gender', 'hiredate', 'jobtitle', 'loginid', 'maritalstatus', 'modifieddate', 'nationalidnumber', 'organizationnode', 'rowguid', 'salariedflag', 'sickleavehours', 'vacationhours', 'hire_year']


In [271]:
for col in raw_data.columns:
    print(f"Valores únicos em '{col}':", raw_data[col].unique()[:10])  # Limitar a exibição a 10 valores



Valores únicos em 'birthdate': <DatetimeArray>
['1969-01-29 00:00:00', '1971-08-01 00:00:00', '1974-11-12 00:00:00', '1974-12-23 00:00:00', '1952-09-27 00:00:00', '1959-03-11 00:00:00', '1987-02-24 00:00:00', '1986-06-05 00:00:00', '1979-01-21 00:00:00', '1984-11-30 00:00:00']
Length: 10, dtype: datetime64[ns]
Valores únicos em 'businessentityid': [ 1  2  3  4  5  6  7  8  9 10]
Valores únicos em 'currentflag': [ True]
Valores únicos em 'gender': ['M' 'F']
Valores únicos em 'hiredate': <DatetimeArray>
['2009-01-14 00:00:00', '2008-01-31 00:00:00', '2007-11-11 00:00:00', '2007-12-05 00:00:00', '2008-01-06 00:00:00', '2008-01-24 00:00:00', '2009-02-08 00:00:00', '2008-12-29 00:00:00', '2009-01-16 00:00:00', '2009-05-03 00:00:00']
Length: 10, dtype: datetime64[ns]
Valores únicos em 'jobtitle': ['Chief Executive Officer' 'Vice President Of Engineering'
 'Engineering Manager' 'Senior Tool Designer' 'Design Engineer'
 'Research And Development Manager' 'Research And Development Engineer'
 'T

In [272]:
# Listar colunas binárias esperadas
binary_columns = ['currentflag', 'salariedflag']

# Verificar valores únicos em colunas binárias
for col in binary_columns:
    unique_values = raw_data[col].unique()
    print(f"Valores únicos em '{col}': {unique_values}")

    # Corrigir valores não binários, se necessário
    if not set(unique_values).issubset({True, False, 0, 1}):
        pr



Valores únicos em 'currentflag': [ True]
Valores únicos em 'salariedflag': [ True False]


In [273]:
# Contar valores em 'currentflag' e 'salariedflag'
print("Distribuição de 'currentflag':")
print(raw_data['currentflag'].value_counts())

print("\nDistribuição de 'salariedflag':")
print(raw_data['salariedflag'].value_counts())


Distribuição de 'currentflag':
currentflag
True    290
Name: count, dtype: int64

Distribuição de 'salariedflag':
salariedflag
False    238
True      52
Name: count, dtype: int64


In [274]:
# 1. Verificar se todos os funcionários ativos têm currentflag = True, pois deveria ser false = demitido/desligado
print("Funcionários ativos errados:", raw_data[raw_data['currentflag'] != True])

# 2. Validar datas
print("Contratações futuras:", raw_data[raw_data['hiredate'] > pd.Timestamp.now()])
print("Modifieddate antes de hiredate:", raw_data[raw_data['modifieddate'] < raw_data['hiredate']])

# 3. Verificar unicidade de identificadores
print("Duplicados em 'businessentityid':", raw_data['businessentityid'].duplicated().sum())
print("Duplicados em 'nationalidnumber':", raw_data['nationalidnumber'].duplicated().sum())



Funcionários ativos errados: Empty DataFrame
Columns: [birthdate, businessentityid, currentflag, gender, hiredate, jobtitle, loginid, maritalstatus, modifieddate, nationalidnumber, organizationnode, rowguid, salariedflag, sickleavehours, vacationhours, hire_year]
Index: []
Contratações futuras: Empty DataFrame
Columns: [birthdate, businessentityid, currentflag, gender, hiredate, jobtitle, loginid, maritalstatus, modifieddate, nationalidnumber, organizationnode, rowguid, salariedflag, sickleavehours, vacationhours, hire_year]
Index: []
Modifieddate antes de hiredate: Empty DataFrame
Columns: [birthdate, businessentityid, currentflag, gender, hiredate, jobtitle, loginid, maritalstatus, modifieddate, nationalidnumber, organizationnode, rowguid, salariedflag, sickleavehours, vacationhours, hire_year]
Index: []
Duplicados em 'businessentityid': 0
Duplicados em 'nationalidnumber': 0


## ESTATÍSTICA DESCRITIVA

In [ ]:
# Selecionar colunas relevantes para análise descritiva
cols_para_analise = ['sickleavehours', 'vacationhours', 'salariedflag']

# Garantir que as datas estejam no formato correto
raw_data['hire_year'] = pd.to_datetime(raw_data['hiredate']).dt.year

# Adicionar a nova coluna à lista
cols_para_analise.append('hire_year')

# Gerar estatísticas descritivas
analise_descritiva = raw_data[cols_para_analise].describe(include='all')

# Substituir NaN por '-'
analise_descritiva = analise_descritiva.fillna('-')

print(analise_descritiva)
